In [122]:
#  importing required libraries 
#import pickle  #to save trained model 
import pandas as pd
import joblib
from sklearn.preprocessing import StandardScaler   #standardize features to improve its performance
from sklearn.pipeline import Pipeline

In [123]:
#ipl = pd.read_csv("data/cleaned/ipl_master.csv")
ipl = pd.read_csv(
    "data/cleaned/ipl_master.csv",
    low_memory=False
)

In [124]:
print(ipl)

        match_id  inning           batting_team                 bowling_team  \
0         335982       1  Kolkata Knight Riders  Royal Challengers Bangalore   
1         335982       1  Kolkata Knight Riders  Royal Challengers Bangalore   
2         335982       1  Kolkata Knight Riders  Royal Challengers Bangalore   
3         335982       1  Kolkata Knight Riders  Royal Challengers Bangalore   
4         335982       1  Kolkata Knight Riders  Royal Challengers Bangalore   
...          ...     ...                    ...                          ...   
260915   1426312       2  Kolkata Knight Riders          Sunrisers Hyderabad   
260916   1426312       2  Kolkata Knight Riders          Sunrisers Hyderabad   
260917   1426312       2  Kolkata Knight Riders          Sunrisers Hyderabad   
260918   1426312       2  Kolkata Knight Riders          Sunrisers Hyderabad   
260919   1426312       2  Kolkata Knight Riders          Sunrisers Hyderabad   

        over  ball       batter        

In [140]:
#------for pvp avg we need bowler but bowler is ball by ball 
ipl = ipl.sort_values(['batter', 'bowler', 'date'])

ipl['pvp_avg'] = (
    ipl.groupby(['batter','bowler'])['batsman_runs']
       .transform(lambda x: x.shift().expanding().mean())
)
print(ipl)


       match_id  inning                 batting_team      bowling_team  over  \
87643    598044       2          Sunrisers Hyderabad  Delhi Daredevils     5   
87644    598044       2          Sunrisers Hyderabad  Delhi Daredevils     5   
87645    598044       2          Sunrisers Hyderabad  Delhi Daredevils     5   
87646    598044       2          Sunrisers Hyderabad  Delhi Daredevils     5   
87648    598044       2          Sunrisers Hyderabad  Delhi Daredevils     5   
...         ...     ...                          ...               ...   ...   
8115     336016       1  Royal Challengers Bangalore   Kings XI Punjab    18   
8116     336016       1  Royal Challengers Bangalore   Kings XI Punjab    18   
8118     336016       1  Royal Challengers Bangalore   Kings XI Punjab    18   
9651     336023       2  Royal Challengers Bangalore  Rajasthan Royals    14   
9652     336023       2  Royal Challengers Bangalore  Rajasthan Royals    14   

       ball          batter     bowler 

In [126]:
ipl.columns

Index(['match_id', 'inning', 'batting_team', 'bowling_team', 'over', 'ball',
       'batter', 'bowler', 'non_striker', 'batsman_runs', 'extra_runs',
       'total_runs', 'extras_type', 'is_wicket', 'player_dismissed',
       'dismissal_kind', 'fielder', 'id', 'season', 'venue', 'date', 'team1',
       'team2', 'pvp_avg'],
      dtype='object')

In [127]:
#____player_match_aggregation
df = ipl.groupby(['date','match_id','batter','venue','bowling_team','batting_team']).agg({'batsman_runs':'sum','is_wicket':'max','pvp_avg':'last'}).reset_index()

In [128]:
#To Sort data in chronological order
df = df.sort_values(['batter','date'])

In [129]:
print(df)

            date  match_id          batter  \
4291  2012-04-29    548346  A Ashish Reddy   
4382  2012-05-04    548352  A Ashish Reddy   
4488  2012-05-08    548359  A Ashish Reddy   
4699  2012-05-18    548373  A Ashish Reddy   
4747  2012-05-20    548376  A Ashish Reddy   
...          ...       ...             ...   
7778  2016-04-10    980903          Z Khan   
8412  2016-05-15    980993          Z Khan   
8687  2017-04-08   1082595          Z Khan   
9287  2017-05-06   1082635          Z Khan   
9452  2017-05-14   1082646          Z Khan   

                                                  venue  \
4291                                   Wankhede Stadium   
4382                    MA Chidambaram Stadium, Chepauk   
4488          Rajiv Gandhi International Stadium, Uppal   
4699          Rajiv Gandhi International Stadium, Uppal   
4747          Rajiv Gandhi International Stadium, Uppal   
...                                                 ...   
7778                              

In [130]:
#__________bowler aggregation
bowler_df = ipl.groupby(
    ['date','match_id','bowler','venue','batting_team','bowling_team']
).agg({
    'total_runs': 'sum',   # runs conceded by bowler
    'is_wicket': 'sum'     # wickets taken by bowler
}).reset_index()

# sort in chronological order per bowler
bowler_df = bowler_df.sort_values(['bowler', 'date'])


In [131]:
print(bowler_df)

            date  match_id          bowler                            venue  \
3373  2012-04-26    548341  A Ashish Reddy       Subrata Roy Sahara Stadium   
3431  2012-04-29    548346  A Ashish Reddy                 Wankhede Stadium   
3457  2012-05-01    548348  A Ashish Reddy                 Barabati Stadium   
3507  2012-05-04    548352  A Ashish Reddy  MA Chidambaram Stadium, Chepauk   
3557  2012-05-06    548356  A Ashish Reddy            M Chinnaswamy Stadium   
...          ...       ...             ...                              ...   
7235  2017-04-28   1082622          Z Khan                     Eden Gardens   
7391  2017-05-06   1082635          Z Khan                 Feroz Shah Kotla   
7451  2017-05-10   1082640          Z Khan                       Green Park   
7477  2017-05-12   1082642          Z Khan                 Feroz Shah Kotla   
7525  2017-05-14   1082646          Z Khan                 Feroz Shah Kotla   

                     batting_team      bowling_team

In [132]:
#______rolling averages (form)feature
df['rolling_avg_10']=df.groupby('batter')['batsman_runs'].rolling(10).mean().reset_index(0,drop=True)
print(df.head())                                                          

            date  match_id          batter  \
4291  2012-04-29    548346  A Ashish Reddy   
4382  2012-05-04    548352  A Ashish Reddy   
4488  2012-05-08    548359  A Ashish Reddy   
4699  2012-05-18    548373  A Ashish Reddy   
4747  2012-05-20    548376  A Ashish Reddy   

                                          venue                 bowling_team  \
4291                           Wankhede Stadium               Mumbai Indians   
4382            MA Chidambaram Stadium, Chepauk          Chennai Super Kings   
4488  Rajiv Gandhi International Stadium, Uppal              Kings XI Punjab   
4699  Rajiv Gandhi International Stadium, Uppal             Rajasthan Royals   
4747  Rajiv Gandhi International Stadium, Uppal  Royal Challengers Bangalore   

         batting_team  batsman_runs  is_wicket  pvp_avg  rolling_avg_10  
4291  Deccan Chargers            10          1     0.25             NaN  
4382  Deccan Chargers             3          1     2.00             NaN  
4488  Deccan Charger

In [133]:
#______rolling averages (form)feature for wickets
bowler_df['rolling_avg_wickets']=(
    bowler_df.groupby('bowler')['is_wicket'].rolling(window=5, min_periods=1).mean().reset_index(0,drop=True)
)
print(bowler_df[['date','match_id','bowler','rolling_avg_wickets']])                                                        

            date  match_id          bowler  rolling_avg_wickets
3373  2012-04-26    548341  A Ashish Reddy             2.000000
3431  2012-04-29    548346  A Ashish Reddy             1.500000
3457  2012-05-01    548348  A Ashish Reddy             1.333333
3507  2012-05-04    548352  A Ashish Reddy             1.250000
3557  2012-05-06    548356  A Ashish Reddy             1.200000
...          ...       ...             ...                  ...
7235  2017-04-28   1082622          Z Khan             0.400000
7391  2017-05-06   1082635          Z Khan             0.400000
7451  2017-05-10   1082640          Z Khan             0.000000
7477  2017-05-12   1082642          Z Khan             0.400000
7525  2017-05-14   1082646          Z Khan             0.800000

[12978 rows x 4 columns]


In [134]:
#df['overs_bowled_last5']=df.groupby('bowler')['over'].rolling(5).count().reset_index(0, drop=True)/6
#print(df['overs_bowled_last5'])

In [135]:
#____________venue averages
df['venue_avg']=df.groupby(['batter','venue'])['batsman_runs'].transform('mean')
print(df)
                

            date  match_id          batter  \
4291  2012-04-29    548346  A Ashish Reddy   
4382  2012-05-04    548352  A Ashish Reddy   
4488  2012-05-08    548359  A Ashish Reddy   
4699  2012-05-18    548373  A Ashish Reddy   
4747  2012-05-20    548376  A Ashish Reddy   
...          ...       ...             ...   
7778  2016-04-10    980903          Z Khan   
8412  2016-05-15    980993          Z Khan   
8687  2017-04-08   1082595          Z Khan   
9287  2017-05-06   1082635          Z Khan   
9452  2017-05-14   1082646          Z Khan   

                                                  venue  \
4291                                   Wankhede Stadium   
4382                    MA Chidambaram Stadium, Chepauk   
4488          Rajiv Gandhi International Stadium, Uppal   
4699          Rajiv Gandhi International Stadium, Uppal   
4747          Rajiv Gandhi International Stadium, Uppal   
...                                                 ...   
7778                              

In [136]:
bowler_df['venue_avg_wickets']=(
    bowler_df.groupby(['bowler','venue'])['is_wicket'].transform('mean')
)
print(bowler_df[['bowler','venue','venue_avg_wickets']])

              bowler                            venue  venue_avg_wickets
3373  A Ashish Reddy       Subrata Roy Sahara Stadium           2.000000
3431  A Ashish Reddy                 Wankhede Stadium           1.000000
3457  A Ashish Reddy                 Barabati Stadium           1.000000
3507  A Ashish Reddy  MA Chidambaram Stadium, Chepauk           0.500000
3557  A Ashish Reddy            M Chinnaswamy Stadium           0.500000
...              ...                              ...                ...
7235          Z Khan                     Eden Gardens           0.666667
7391          Z Khan                 Feroz Shah Kotla           1.428571
7451          Z Khan                       Green Park           0.000000
7477          Z Khan                 Feroz Shah Kotla           1.428571
7525          Z Khan                 Feroz Shah Kotla           1.428571

[12978 rows x 3 columns]


In [137]:
#_____________opponent-specific stats(Player vs Team)
#df['pvt_avg'] = df.groupby(['batter','bowler'])['batsman_runs'].transform('mean')

#___shift(1) avoids data leakage____

df['pvt_avg'] = df.groupby(['batter','bowling_team'])['batsman_runs']\
                           .expanding().mean().shift(1)\
                           .reset_index(level=[0,1], drop=True)
print(df)

            date  match_id          batter  \
4291  2012-04-29    548346  A Ashish Reddy   
4382  2012-05-04    548352  A Ashish Reddy   
4488  2012-05-08    548359  A Ashish Reddy   
4699  2012-05-18    548373  A Ashish Reddy   
4747  2012-05-20    548376  A Ashish Reddy   
...          ...       ...             ...   
7778  2016-04-10    980903          Z Khan   
8412  2016-05-15    980993          Z Khan   
8687  2017-04-08   1082595          Z Khan   
9287  2017-05-06   1082635          Z Khan   
9452  2017-05-14   1082646          Z Khan   

                                                  venue  \
4291                                   Wankhede Stadium   
4382                    MA Chidambaram Stadium, Chepauk   
4488          Rajiv Gandhi International Stadium, Uppal   
4699          Rajiv Gandhi International Stadium, Uppal   
4747          Rajiv Gandhi International Stadium, Uppal   
...                                                 ...   
7778                              

In [138]:
#_____________opponent-specific stats(Player vs player(bowler)
#df['pvp_avg'] = df.groupby(['batter','bowler'])['batsman_runs']\
 #                          .expanding().mean().shift(1)\
  #                         .reset_index(level=[0,1], drop=True)
#print(df.head())

In [141]:
bowler_df['opponent_wickets']=(bowler_df.groupby(['bowler','batting_team'])['is_wicket'].transform('mean'))
print(bowler_df)

            date  match_id          bowler                            venue  \
3373  2012-04-26    548341  A Ashish Reddy       Subrata Roy Sahara Stadium   
3431  2012-04-29    548346  A Ashish Reddy                 Wankhede Stadium   
3457  2012-05-01    548348  A Ashish Reddy                 Barabati Stadium   
3507  2012-05-04    548352  A Ashish Reddy  MA Chidambaram Stadium, Chepauk   
3557  2012-05-06    548356  A Ashish Reddy            M Chinnaswamy Stadium   
...          ...       ...             ...                              ...   
7235  2017-04-28   1082622          Z Khan                     Eden Gardens   
7391  2017-05-06   1082635          Z Khan                 Feroz Shah Kotla   
7451  2017-05-10   1082640          Z Khan                       Green Park   
7477  2017-05-12   1082642          Z Khan                 Feroz Shah Kotla   
7525  2017-05-14   1082646          Z Khan                 Feroz Shah Kotla   

                     batting_team      bowling_team

In [157]:
#bowler_df['bowler_economy']=(
 # bowler_df.groupby(['match_id','bowler'])['total_runs'].transform('sum')/
  #  (
   ##    .transform('nunique')
   #)
#)
#bowler_df['bowler_economy'] = (
 #   bowler_df['total_runs'] / bowler_df['overs_bowled']
#)
#bowler_df.rename(columns={'total_runs':'runs_conceded'}, inplace=True)
#print(bowler_df[['date','match_id','bowler','bowler_economy']])

KeyError: 'overs_bowled'

In [143]:
#______________career stats
df['career_avg'] = df.groupby('batter')['batsman_runs']\
                           .expanding().mean().shift(1)\
                           .reset_index(level=0, drop=True)
print(df.head())


            date  match_id          batter  \
4291  2012-04-29    548346  A Ashish Reddy   
4382  2012-05-04    548352  A Ashish Reddy   
4488  2012-05-08    548359  A Ashish Reddy   
4699  2012-05-18    548373  A Ashish Reddy   
4747  2012-05-20    548376  A Ashish Reddy   

                                          venue                 bowling_team  \
4291                           Wankhede Stadium               Mumbai Indians   
4382            MA Chidambaram Stadium, Chepauk          Chennai Super Kings   
4488  Rajiv Gandhi International Stadium, Uppal              Kings XI Punjab   
4699  Rajiv Gandhi International Stadium, Uppal             Rajasthan Royals   
4747  Rajiv Gandhi International Stadium, Uppal  Royal Challengers Bangalore   

         batting_team  batsman_runs  is_wicket  pvp_avg  rolling_avg_10  \
4291  Deccan Chargers            10          1     0.25             NaN   
4382  Deccan Chargers             3          1     2.00             NaN   
4488  Deccan Char

In [142]:
bowler_df['bowler_career_wickets_avg']=bowler_df.groupby('bowler')['is_wicket']\
                           .expanding().mean().shift(1)\
                           .reset_index(level=0, drop=True)
print(bowler_df.head(6))


            date  match_id          bowler  \
3373  2012-04-26    548341  A Ashish Reddy   
3431  2012-04-29    548346  A Ashish Reddy   
3457  2012-05-01    548348  A Ashish Reddy   
3507  2012-05-04    548352  A Ashish Reddy   
3557  2012-05-06    548356  A Ashish Reddy   
3593  2012-05-08    548359  A Ashish Reddy   

                                          venue                 batting_team  \
3373                 Subrata Roy Sahara Stadium                Pune Warriors   
3431                           Wankhede Stadium               Mumbai Indians   
3457                           Barabati Stadium                Pune Warriors   
3507            MA Chidambaram Stadium, Chepauk          Chennai Super Kings   
3557                      M Chinnaswamy Stadium  Royal Challengers Bangalore   
3593  Rajiv Gandhi International Stadium, Uppal              Kings XI Punjab   

         bowling_team  total_runs  is_wicket  rolling_avg_wickets  \
3373  Deccan Chargers          32          2   

In [144]:
#___________Create target label
df['next_match_runs']=df.groupby('batter')['batsman_runs'].shift(-1)
df=df.dropna()
print(df.head())

            date  match_id          batter  \
5089  2013-04-14    598013  A Ashish Reddy   
5173  2013-04-17    598018  A Ashish Reddy   
5226  2013-04-19    598021  A Ashish Reddy   
5357  2013-04-25    598030  A Ashish Reddy   
5386  2013-04-27    598032  A Ashish Reddy   

                                          venue           bowling_team  \
5089                               Eden Gardens  Kolkata Knight Riders   
5173    Maharashtra Cricket Association Stadium          Pune Warriors   
5226  Rajiv Gandhi International Stadium, Uppal        Kings XI Punjab   
5357            MA Chidambaram Stadium, Chepauk    Chennai Super Kings   
5386                     Sawai Mansingh Stadium       Rajasthan Royals   

             batting_team  batsman_runs  is_wicket   pvp_avg  rolling_avg_10  \
5089  Sunrisers Hyderabad             4          1  1.000000             7.9   
5173  Sunrisers Hyderabad            19          0  1.000000             8.8   
5226  Sunrisers Hyderabad             

In [145]:
df["next_match_runs"].value_counts().head(10)


next_match_runs
0.0    858
1.0    669
2.0    466
4.0    392
5.0    353
6.0    352
8.0    341
3.0    323
9.0    304
7.0    299
Name: count, dtype: int64

In [146]:
df["next_match_runs"].describe()


count    11689.000000
mean        22.310292
std         22.247372
min          0.000000
25%          5.000000
50%         15.000000
75%         33.000000
max        175.000000
Name: next_match_runs, dtype: float64

In [148]:
bowler_df['next_match_wicket'] = bowler_df.groupby('bowler')['is_wicket'].shift(-1)
bowler_df = bowler_df.dropna()
print(bowler_df.head(5))

            date  match_id          bowler  \
3431  2012-04-29    548346  A Ashish Reddy   
3457  2012-05-01    548348  A Ashish Reddy   
3507  2012-05-04    548352  A Ashish Reddy   
3557  2012-05-06    548356  A Ashish Reddy   
3593  2012-05-08    548359  A Ashish Reddy   

                                          venue                 batting_team  \
3431                           Wankhede Stadium               Mumbai Indians   
3457                           Barabati Stadium                Pune Warriors   
3507            MA Chidambaram Stadium, Chepauk          Chennai Super Kings   
3557                      M Chinnaswamy Stadium  Royal Challengers Bangalore   
3593  Rajiv Gandhi International Stadium, Uppal              Kings XI Punjab   

         bowling_team  total_runs  is_wicket  rolling_avg_wickets  \
3431  Deccan Chargers          11          1             1.500000   
3457  Deccan Chargers          32          1             1.333333   
3507  Deccan Chargers          16  

In [151]:
## _________________Feature selection for Batsman
features=df[['rolling_avg_10','venue_avg','pvt_avg','pvp_avg','career_avg']]
labels  =df[['next_match_runs']]
print(features.head(10))
print(labels)

              batter  rolling_avg_10  venue_avg    pvt_avg   pvp_avg  \
5089  A Ashish Reddy             7.9   4.000000  12.333333  1.000000   
5173  A Ashish Reddy             8.8  19.000000   7.000000  1.000000   
5226  A Ashish Reddy             9.2   8.454545   8.000000  1.666667   
5357  A Ashish Reddy            12.0  19.500000   3.000000  6.000000   
5386  A Ashish Reddy            12.4  14.000000  10.000000  1.333333   
5555  A Ashish Reddy            12.5   8.454545  16.000000  0.000000   
7017  A Ashish Reddy            13.1  14.000000  12.000000  2.333333   
7044  A Ashish Reddy            13.2  14.000000  10.500000  1.000000   
7245  A Ashish Reddy            15.1  22.000000   7.500000  6.000000   
7346  A Ashish Reddy            14.1   8.454545  19.500000  2.000000   

      career_avg  
5089    8.333333  
5173    7.900000  
5226    8.909091  
5357    8.750000  
5386   10.846154  
5555   11.071429  
7017   10.666667  
7044   10.812500  
7245   11.058824  
7346   11.666667 

In [149]:
print(df.columns.tolist())

['date', 'match_id', 'batter', 'venue', 'bowling_team', 'batting_team', 'batsman_runs', 'is_wicket', 'pvp_avg', 'rolling_avg_10', 'venue_avg', 'pvt_avg', 'career_avg', 'next_match_runs']


In [160]:
##_________________________feature selection for bowler
features = bowler_df[['rolling_avg_wickets','venue_avg_wickets','opponent_wickets','bowler_career_wickets_avg']]
labels = bowler_df[['next_match_wicket']]
print('Bowler Features:')
print(features.head())
print(labels)

Bowler Features:
      rolling_avg_wickets  venue_avg_wickets  opponent_wickets  \
3431             1.500000               1.00          1.000000   
3457             1.333333               1.00          1.333333   
3507             1.250000               0.50          1.000000   
3557             1.200000               0.50          1.000000   
3593             1.200000               1.25          2.000000   

      bowler_career_wickets_avg  
3431                   2.000000  
3457                   1.500000  
3507                   1.333333  
3557                   1.250000  
3593                   1.200000  
      next_match_wicket
3431                1.0
3457                1.0
3507                1.0
3557                2.0
3593                0.0
...                 ...
7153                0.0
7235                0.0
7391                0.0
7451                2.0
7477                2.0

[12447 rows x 1 columns]


In [170]:
# sort chronologically for batsman
df = df.sort_values('date')

# features and label
X_bat = df[['rolling_avg_10', 'venue_avg', 'pvt_avg', 'pvp_avg', 'career_avg']]
y_bat = df[['next_match_runs']]

# time-based split
split_bat = int(len(df) * 0.8)

X_train_bat = X_bat.iloc[:split_bat]
X_test_bat  = X_bat.iloc[split_bat:]

y_train_bat = y_bat.iloc[:split_bat]
y_test_bat  = y_bat.iloc[split_bat:]

print(X_train_bat.shape, X_test_bat.shape)
print(y_train_bat.shape, y_test_bat.shape)


(9351, 5) (2338, 5)
(9351, 1) (2338, 1)


In [171]:
# sort chronologically for bowler
bowler_df = bowler_df.sort_values('date')
# features and label
X_bowl = bowler_df[['rolling_avg_wickets','venue_avg_wickets','opponent_wickets','bowler_career_wickets_avg']]
y_bowl = bowler_df[['next_match_wicket']]

# time-based split
split_bowl = int(len(bowler_df) * 0.8)

X_train_bowl = X_bowl.iloc[:split_bowl]
X_test_bowl  = X_bowl.iloc[split_bowl:]

y_train_bowl = y.iloc[:split_bowl]
y_test_bowl  = y.iloc[split_bowl:]

print(X_train_bowl.shape, X_test_bowl.shape)
print(y_train_bowl.shape, y_test_bowl.shape)


(9957, 4) (2490, 4)
(9957, 1) (2490, 1)


In [173]:
#     Feature pipeline
pipeline_bat = Pipeline([
    ('scaler', StandardScaler())
])

pipeline_bowl = Pipeline([
    ('scaler', StandardScaler())
])

pipeline_bat.fit(X_train_bat)
pipeline_bowl.fit(X_train_bowl)

joblib.dump(pipeline_bat, "scripts/batter_pipeline.pkl")
joblib.dump(pipeline_bowl, "scripts/bowler_pipeline.pkl")


['scripts/bowler_pipeline.pkl']

In [174]:
#____________save Final Dataset
#final_df = df[['batter','date','venue','bowler','bowling_team','rolling_avg_10','venue_avg','pvt_avg','pvp_avg','career_avg','next_match_runs','next_match_wicket','rolling_avg_wickets',
 #       'venue_avg_wickets','opponent_wickets','bowler_career_wickets_avg']]
#print(final_df.head())

KeyError: "['bowler', 'next_match_wicket', 'rolling_avg_wickets', 'venue_avg_wickets', 'opponent_wickets', 'bowler_career_wickets_avg'] not in index"

In [177]:
# for batter
df_batter = df[[
    'match_id',
    'date',
    'batter',
    'batting_team',
    'bowling_team',
    'venue',
    'rolling_avg_10',
    'venue_avg',
    'pvt_avg',
    'pvp_avg',
    'career_avg',
    'next_match_runs'
]]

df_batter.to_csv("data/processed/dataset.csv", index=False)

# for bowler
df_bowler = bowler_df[[
    'match_id',
    'date',
    'bowler',
    'bowling_team',
    'batting_team',
    'venue',
    'rolling_avg_wickets',
    'venue_avg_wickets',
    'opponent_wickets',
    'bowler_career_wickets_avg',
    'next_match_wicket'
]]

df_bowler.to_csv("data/processed/bowler_dataset.csv", index=False)



In [178]:
#final_df.to_csv("data/processed/dataset.csv",index=False)
print("Milestone-2 is completed:dataset.csv & feature_pipeline.pkl created!")

Milestone-2 is completed:dataset.csv & feature_pipeline.pkl created!


In [179]:
print("Max next_match_runs:", df["next_match_runs"].max())
print(df["next_match_runs"].describe())

print("Max rolling_avg_10:", df["rolling_avg_10"].max())
print(df["rolling_avg_10"].describe())


Max next_match_runs: 175.0
count    11689.000000
mean        22.310292
std         22.247372
min          0.000000
25%          5.000000
50%         15.000000
75%         33.000000
max        175.000000
Name: next_match_runs, dtype: float64
Max rolling_avg_10: 69.0
count    11689.000000
mean        22.696587
std         10.205279
min          0.600000
25%         15.400000
50%         22.300000
75%         29.000000
max         69.000000
Name: rolling_avg_10, dtype: float64
